In [ ]:
import pandas as pd
import pyodbc
import sqlite3

In [ ]:
DB = {'servername': r'DESKTOP-I3QFF0F\SQLEXPRESS', 'database': 'DWH'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

go_sales_connect = sqlite3.connect('go_sales.sqlite')
go_crm_connect = sqlite3.connect('go_crm.sqlite')
go_staff_connect = sqlite3.connect('go_staff.sqlite')

In [ ]:
order = pd.read_sql_query("SELECT * FROM satisfaction", go_staff_connect )
order

In [ ]:
for index, row in order.iterrows():
    query_satisfaction = f"INSERT INTO SATISFACTION VALUES ({row['SATISFACTION_YEAR_number']},'{row['SATISFACTION_STAFF_id']}', '{row['SATISFACTION_TYPE_id']}', '{row['SATISFACTION_TYPE_description']}')"
    export_cursor.execute(query_satisfaction)
   

export_conn.commit()
export_cursor.close()